# Hekdesh - רשם ההקדשים
This notebook analyzes data from [רשות התאגידים - רשם ההקדשות](https://data.gov.il/dataset/hekdeshot)  and generates a dataflow-pipeline. For addtional information see [OpenBudget-card](https://github.com/orgs/OpenBudget/projects/3#card-31241405).  

מאגר מידע זה מכיל שלושה קבצים אודות הקדשות ציבוריים: האחד כולל מידע כללי על הקדשות ציבוריים הרשומים פנקס ההקדשות הציבוריים; הקובץ השני כולל מידע אודות יתרת הנכסים של הקדשות ציבוריים; הקובץ השלישי כולל מידע אודות מקרקעין השייכים להקדשות ציבוריים.
קבצי מידע

    רשם ההקדשות - מידע כלליCSV
    תאריך עדכון: 1 מרץ, 2020
    מספר צריכות: 637
    מספר הורדות: 2

    אפשרויות
    רשם ההקדשות - מקרקעין של הקדש ציבוריCSV
    תאריך עדכון: 23 פברואר, 2020
    מספר הורדות: 1

    אפשרויות
    רשם ההקדשות - יתרת הנכסים של הקדש ציבוריCSV
    תאריך עדכון: 23 פברואר, 2020
    מספר צריכות: 379



In [38]:
!pip install dataflows

In [5]:
%load_ext blackcellmagic

In [35]:
import pandas as pd
from dataflows import (
    Flow,
    load,
    set_type,
    printer,
    join,
    checkpoint,
    concatenate,
    join_with_self,
    add_field,
    delete_fields,
)

In [3]:
# a stale version of the datasets were loaded to github
hekdeshGeneral = r"https://raw.githubusercontent.com/sephib/budgetkey-data-pipelines/master/data/hekdesh/hekdeshGeneral.csv"
hekdeshProperty = r"https://raw.githubusercontent.com/sephib/budgetkey-data-pipelines/master/data/hekdesh/hekdeshPropery.csv"
hekdeshFinance =  r"https://raw.githubusercontent.com/sephib/budgetkey-data-pipelines/master/data/hekdesh/hekdeshFinance.csv"  #hekdeshot_nechasimjustice.gov.il.csv"  #r"../data/hekdesh/hekdeshFinance.csv"

## Hekdesh General

In [ ]:
df = pd.read_csv(
    hekdeshGeneral,
    names=["hekdesh_id", "record_date", "name", "purpose", "funding", "status", "col1"],
    dtype={
        "hekdesh_id": int,
        "record_date": object,
        "name": object,
        "purpose": object,
        "funding": "category",
        "status": "category",
        "col1": object,
    },
    skiprows=1,
    usecols=["hekdesh_id", "record_date", "name", "purpose", "funding", "status",],
)
df["record_date"] = pd.to_datetime(df["record_date"], format="%d/%m/%Y")
# df = df.set_index("hekdesh_id")
df.head(1)

In [15]:
df['hekdesh_id'].is_unique

True

## Hekdesh Property
* each hekdesh can have sevral properties

In [12]:
df_p = pd.read_csv(
    hekdeshProperty,
    names=[
        "hekdesh_id",
        "name",
        "description",
        "gush",
        "chelka",
        "tat_chelka",
        "notes",
        "col1",
    ],
    dtype={
        "hekdesh_id": int,
        "name": object,
        "description": "category",
        "guhs": int,
        "chelcka": int,
        "tat_chelka": int,
        "notes": object,
        "col1": object,
    },
    usecols=[
        "hekdesh_id",
        "name",
        "description",
        "gush",
        "chelka",
        "tat_chelka",
        "notes",
    ],
    skiprows=1,
)
df_p.head()

,hekdesh_id,name,description,gush,chelka,tat_chelka,notes
0,590000014,יעקבזון דניאל קרן על שמו,מגרש+בית,6188,34,0,יש
1,590000022,"בן חיים בת שבע ומרדכי , קרן לגמ~ח לחינוך ולתרב...",דירה,6125,516,0,אין
2,590000022,"בן חיים בת שבע ומרדכי , קרן לגמ~ח לחינוך ולתרב...",מלון,30042,60,0,אין
3,590000022,"בן חיים בת שבע ומרדכי , קרן לגמ~ח לחינוך ולתרב...",גג בניין,30042,60,0,אין
4,590000030,"אוקס ד~ר דוד וגרדה, קרן על שמם",דירה,30007,42,12,יש


## Hekdesh Finance
* each hekdesh can have has several finance records
* some records have value=0 - not sure if relevant
* some Hekdesh can have several values - even in the same currency (even without dates)- TBD

In [19]:
df_f = pd.read_csv(
    hekdeshFinance,
    names=["hekdesh_id", "name", "currency", "value", "last_date_update", "col3"],
    dtype={
        "hekdesh_id": int,
        "name": object,
        "col1": "category",
        "col2": int,
        "last_date_update": object,
        "col3": object,
    },
    skiprows=1,
    usecols=["hekdesh_id", "name", "currency", "value", "last_date_update"],
    encoding="utf-8",
)
df_f["last_date_update"] = pd.to_datetime(df_f["last_date_update"], format="%d/%m/%Y")
# df_f = df_f.set_index('hekdesh_id')
df_f.head()

,hekdesh_id,name,currency,value,last_date_update
0,590000014,יעקבזון דניאל קרן על שמו,ש~ח,210378,2011-12-31
1,590000030,"אוקס ד~ר דוד וגרדה, קרן על שמם",ש~ח,707908,2018-12-31
2,590000048,זכרון מני סימי,ש~ח,374033,2006-12-31
3,590000089,לבנטל אברהם ובלה קרן על שמם,ש~ח,1045978,2018-12-31
4,590036000,קרן ע~ש רבקה לאה ביילין ז~ל בבי~ס לרפואה הדסה ...,ש~ח,1,2012-11-19


In [22]:
# Each Hekdesh can have several values - even in the same currency (even without dates)- TBD
df_f[df_f["hekdesh_id"].groupby(df_f["hekdesh_id"]).transform("size") > 1]

,hekdesh_id,name,currency,value,last_date_update
53,590000378,לחובסקי אליעזר ריבקינד יצחק ז~ל הקדש ע~ש,ש~ח,21184,NaT
54,590000378,לחובסקי אליעזר ריבקינד יצחק ז~ל הקדש ע~ש,ש~ח,16221,NaT
60,590000501,סלע יצחק ז~ל קרן מילגות ע~ש,NaN,0,NaT
61,590000501,סלע יצחק ז~ל קרן מילגות ע~ש,ש~ח,0,NaT
63,590000527,רובינשטיין בינימין אבימור משה רובינשטיין קרן מ...,NaN,0,NaT
...,...,...,...,...,...
3430,590038063,קרן ג'ול ואלאונור וואקי,ש~ח,316009,2017-06-30
3431,590038063,קרן ג'ול ואלאונור וואקי,אירו,134495,2017-06-30
3487,590037206,"הק' ע~ש דוד ומריאטה חיימוביצ'י,ז'וזף ורוזלי בז...",דולר ארה~ב,222824,2016-05-12
3488,590037206,"הק' ע~ש דוד ומריאטה חיימוביצ'י,ז'וזף ורוזלי בז...",אירו,37828,2016-05-12


In [29]:
df_f[(df_f["hekdesh_id"].groupby(df_f["hekdesh_id"]).transform("size") > 2) &(~df_f["last_date_update"].isnull())]

,hekdesh_id,name,currency,value,last_date_update
556,590007019,כץ אליס מאיר ז~ל קרן מילגות לזיכרו.,דולר ארה~ב,0,1986-03-07
557,590007019,כץ אליס מאיר ז~ל קרן מילגות לזיכרו.,דולר ארה~ב,14932,1986-03-07
558,590007019,כץ אליס מאיר ז~ל קרן מילגות לזיכרו.,שקל (ישן),0,1986-03-07
1416,590015764,קלמן רוסו אירמה רות ודוד קלמן ז~ל קרן לזכרם.,דולר ארה~ב,95678,1993-07-23
1417,590015764,קלמן רוסו אירמה רות ודוד קלמן ז~ל קרן לזכרם.,לי~ש,482,1993-07-23
1418,590015764,קלמן רוסו אירמה רות ודוד קלמן ז~ל קרן לזכרם.,מ~ג,6381,1993-07-23
2486,590027249,גרוס לבינסון מריה ז~ל קרן ע~ש,אירו,44944,2008-12-19
2487,590027249,גרוס לבינסון מריה ז~ל קרן ע~ש,ש~ח,471233,2008-12-19
2488,590027249,גרוס לבינסון מריה ז~ל קרן ע~ש,דולר ארה~ב,188306,2008-12-19
2489,590027249,גרוס לבינסון מריה ז~ל קרן ע~ש,לי~ש,92907,2008-12-19


In [30]:
df_f[df_f["value"]==0]

,hekdesh_id,name,currency,value,last_date_update
18,590037495,הקדש ע~ש שושנה סלטון ז~ל,ש~ח,0,2016-12-31
25,590000139,בלך מריאן ז~לח קרן נאמנות להשכלה גבוהה,ש~ח,0,NaT
49,590000295,ראש הממשלה קרן מלגות,ל~י,0,NaT
55,590000386,פרישאור רוזה ותיאודור קרן,ש~ח,0,2017-12-31
60,590000501,סלע יצחק ז~ל קרן מילגות ע~ש,NaN,0,NaT
...,...,...,...,...,...
3497,590037289,הקדש ע~ש עזרא שררה ז~ל,ש~ח,0,NaT
3510,590036869,הקדש ע~ש בית הכנסת ~תפארת ישראל~,ש~ח,0,2017-12-31
3511,590037222,קרן מלגות ע~ש עמרי סרלין ז~ל ומשפחתו,ש~ח,0,2018-04-01
3515,590037248,הקדש ע~ש רות וולף ז~ל-בחברה להגנת הטבע,ש~ח,0,2017-12-31


# Working with Dataflow

In [44]:
# hekdesh Property
COLUMN_HEADERS_MAPPER_HEKDESH_PROPERTY = {
    "hekdesh_id_property": ["מספר תיק"],
    "hekdesh_name_property": ["שם הקדש"],
    "hekdesh_description": ["תאור הנכס"],
    "gush": ["גוש"],
    "chelcka": ["חלקה"],
    "tat_chelka": ["תת חלקה"],
    "note": ["הערה במרשם המקרקעין על קיומו של הקדש"],
}
hekdesh_property_col = ["hekdesh_description", "gush", "chelcka", "tat_chelka", "note"]
flow_property = Flow(
    load(hekdeshProperty, cast_strategy=load.INFER_STRINGS, name="hekdesh-property"),
    concatenate(
        COLUMN_HEADERS_MAPPER_HEKDESH_PROPERTY,
        target=dict(name="hekdesh-property"),
        resources="hekdesh-property",
    ),
    add_field(
        "hekdesh_property",
        "object",
        lambda row: dict((k, row[k]) for k in hekdesh_property_col),
        resources="hekdesh-property",
    ),
    delete_fields(
        hekdesh_property_col + ["hekdesh_name_property"], resources="hekdesh-property"
    ),    
    printer(num_rows=1),
)
flow_property.process()[1]

hekdesh-property:
#       hekdesh_id_property  hekdesh_property
                  (integer)  (object)
----  ---------------------  --------------------------------------------------------------------------------------------------------
1                 590000014  {'hekdesh_description': 'מגרש+בית', 'gush': '6188', 'chelcka': '34', 'tat_chelka': '0', 'note': 'יש' ...
2                 590000022  {'hekdesh_description': 'דירה', 'gush': '6125', 'chelcka': '516', 'tat_chelka': '0', 'note': 'אין'}
...
1672              590035994  {'hekdesh_description': 'גג בניין', 'gush': '30121', 'chelcka': '17', 'tat_chelka': '0', 'note': 'אי ...


{}

In [57]:
# hekdesh finance
COLUMN_HEADERS_MAPPER_HEKDESH_FINANCE = {
    "currency": ["תאור מטבע"],
    "hekdesh_id_finance": ["מספר תיק"],
    "hekdesh_name_finance": ["שם הקדש"],
    "last_date_value_update": ["נכון לתאריך"],
    "value": ["סכום"],
}
hekdesh_finance_col = [
    "currency",
    "value",
    "last_date_value_update",
]
flow_finance = Flow(
    load(hekdeshFinance, cast_strategy=load.INFER_STRINGS, name="hekdesh-finance"),
    concatenate(
        COLUMN_HEADERS_MAPPER_HEKDESH_FINANCE,
        target=dict(name="hekdesh-finance"),
        resources=["hekdesh-finance"],
    ),
    add_field(
        "hekdesh_finance",
        "object",
        lambda row: dict((k, row[k]) for k in hekdesh_finance_col),
        resources=["hekdesh-finance"],
    ),
    delete_fields(
        hekdesh_finance_col + ["hekdesh_name_finance"], resources=["hekdesh-finance"]
    ),
    printer(num_rows=1),
)
flow_finance.process()[1]

hekdesh-finance:
#       hekdesh_id_finance  hekdesh_finance
                 (integer)  (object)
----  --------------------  -------------------------------------------------------------------------------
1                590000014  {'currency': 'ש~ח', 'value': '210378', 'last_date_value_update': '31/12/2011'}
2                590000030  {'currency': 'ש~ח', 'value': '707908', 'last_date_value_update': '31/12/2018'}
...
3607             590036398  {'currency': 'ש~ח', 'value': '1508165', 'last_date_value_update': '31/12/2018'}


{}

In [58]:
# Hekdesh General
COLUMN_HEADERS_MAPPER_HEKDESH_GENERAL = {
    "hekdesh_funding": ["שיטת מימון"],
    "hekdesh_id": ["מספר תיק"],
    "hekdesh_name": ["שם הקדש"],
    "hekdesh_purpose": ["מטרות"],
    "hekdesh_record_date": ["תאריך רישום"],
    "hekdesh_status": ["סטטווס תיק"],
}
flow_hekdesh = Flow(
    load(hekdeshGeneral, cast_strategy=load.INFER_STRINGS, name="hekdesh-general"),
    concatenate(
        COLUMN_HEADERS_MAPPER_HEKDESH_GENERAL,
        target=dict(name="hekdesh-general"),
        resources=["hekdesh-general"],
    ),
    printer(num_rows=1),
)
flow_hekdesh.process()[1]

hekdesh-general:
#       hekdesh_id  hekdesh_record_date    hekdesh_name                                hekdesh_purpose                                                                                           hekdesh_funding    hekdesh_status
         (integer)  (string)               (string)                                    (string)                                                                                                  (string)           (string)
----  ------------  ---------------------  ------------------------------------------  --------------------------------------------------------------------------------------------------------  -----------------  ----------------
1        590000030  21/02/1974             אוקס ד~ר דוד וגרדה, קרן על שמם              תמיכה בצעירים וצעירות דתיים המכשירים עצמם לתפקידי חינוך או הוראה,תמיכה בבית החולים ליכטנשטדטר בתל אב ...  הכנסה שוטפת        פעיל
2        590000048  23/10/1970             זכרון מני סימי                              יש

{}

In [55]:
flow_joined = Flow(
    load(hekdeshFinance, cast_strategy=load.INFER_STRINGS, name="hekdesh-finance"),
    concatenate(
        COLUMN_HEADERS_MAPPER_HEKDESH_FINANCE,
        target=dict(name="hekdesh-finance"),
        resources=["hekdesh-finance"],
    ),
    add_field(
        "hekdesh_finance_obj_",
        "object",
        lambda row: dict((k, row[k]) for k in hekdesh_finance_col),
        resources="hekdesh-finance",
    ),
    delete_fields(
        hekdesh_finance_col + ["hekdesh_name_finance"], resources="hekdesh-finance"
    ),
    join_with_self(
        "hekdesh-finance",
        ["hekdesh_id_finance"],
        dict(hekdesh_id_finance=None, hekdesh_finance_obj_=dict(aggregate="array")),
    ),
    concatenate(
        dict(
            hekdesh_id2=["hekdesh_id_finance"],
            hekdesh_finance_obj=["hekdesh_finance_ob_"],
        ),
        target=dict(name="hekdesh-finance"),
        resources=["hekdesh-finance"],
    ),
    load(hekdeshProperty, cast_strategy=load.INFER_STRINGS, name="hekdesh-property"),
    load(hekdeshGeneral, cast_strategy=load.INFER_STRINGS, name="hekdesh-general"),    
    concatenate(
        COLUMN_HEADERS_MAPPER_HEKDESH_PROPERTY,
        target=dict(name="hekdesh-property"),
        resources=["hekdesh-property"],
    ),
    add_field(
        "hekdesh_property_obj",
        "object",
        lambda row: dict((k, row[k]) for k in hekdesh_property_col),
        resources="hekdesh-property",
    ),
    delete_fields(
        hekdesh_property_col + ["hekdesh_name_property"], resources=["hekdesh-property"]
    ),
    join_with_self(
        "hekdesh-property",
        ["hekdesh_id_property"],
        dict(hekdesh_id_property=None, hekdesh_property_obj=dict(aggregate="array")),
    ),
    concatenate(
        COLUMN_HEADERS_MAPPER_HEKDESH_GENERAL,
        target=dict(name="hekdesh-general"),
        resources=["hekdesh-general"],
    ),
    join(
        "hekdesh-property",
        ["hekdesh_id_property"],  # Source resource
        "hekdesh-general",
        ["hekdesh_id"],          # Target resource
        fields=({"*": {"aggregate": "first"},}),
        mode="half-outer",
    ),
    delete_fields(["hekdesh_id_property"], resources=["hekdesh-general"]),
    join(
        "hekdesh-finance",
        ["hekdesh_id2"],  # Source resource
        "hekdesh-general",
        ["hekdesh_id"],  # Target resource
        fields=({"*": {"aggregate": "first"},}),
        mode="half-outer",
    ),
    delete_fields(["hekdesh_id2"], resources=["hekdesh-general"]),
    printer(num_rows=1),
)
flow_joined.process()[1]

hekdesh-general:
#       hekdesh_id  hekdesh_record_date    hekdesh_name                                hekdesh_purpose                                                                                           hekdesh_funding    hekdesh_status    hekdesh_property_obj                                                                                      hekdesh_finance_obj
         (integer)  (string)               (string)                                    (string)                                                                                                  (string)           (string)          (array)                                                                                                   (string)
----  ------------  ---------------------  ------------------------------------------  --------------------------------------------------------------------------------------------------------  -----------------  ----------------  ----------------------------------------------------

{}